In [2]:
import findspark
findspark.init()

import io
import re
import os.path as pth
import itertools
import zipfile as zf
import pandas as pd
import urllib.request as ureq
import datetime as dt

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml import Transformer, Pipeline

from utils import TimestampTransformer, GeohashTransformer

### Get weatherstations in/around Stuttgart

In [2]:
lat_bounds = (48.3, 49.1)
lon_bounds = (8.9, 9.5)
#stat_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/historical/TU_Stundenwerte_Beschreibung_Stationen.txt"
stat_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/recent/TU_Stundenwerte_Beschreibung_Stationen.txt"
col_names = ["Stations_id", "von_datum",  "bis_datum" , "Stationshoehe", "geoBreite", "geoLaenge", "Stationsname", "Bundesland"]
df = pd.read_fwf(stat_url, skiprows=2, names=col_names, delim_whitespace=True, error_bad_lines=False, encoding="iso-8859-1")

In [3]:
stgt_stats_df = df[(df["geoBreite"] > lat_bounds[0]) & (df["geoBreite"] < lat_bounds[1]) & 
               (df["geoLaenge"] > lon_bounds[0]) & (df["geoLaenge"] < lon_bounds[1])]
stgt_stats_df

,Stations_id,von_datum,bis_datum,Stationshoehe,geoBreite,geoLaenge,Stationsname,Bundesland
194,2074,20040601,20180109,522,48.3751,8.9801,Hechingen,Baden-Württemberg
315,3278,20090401,20180109,355,48.5376,9.2733,Metzingen,Baden-Württemberg
330,3402,20021101,20180109,750,48.3851,9.4837,Münsingen-Apfelstetten,Baden-Württemberg
359,3671,19790101,19810401,280,48.6321,9.3261,Nürtingen,Baden-Württemberg
391,4160,20040701,20180109,478,48.7425,8.9240,Renningen-Ihinger Hof,Baden-Württemberg
402,4294,19820101,19970101,360,48.4704,8.9687,Rottenburg-Kiebingen,Baden-Württemberg
409,4349,20040601,20180109,248,48.9569,9.0710,Sachsenheim,Baden-Württemberg
450,4926,19790401,20130514,224,48.7896,9.2167,Stuttgart (Neckartal),Baden-Württemberg
451,4927,19760101,19840801,286,48.7693,9.1814,Stuttgart-Stadt,Baden-Württemberg
452,4928,19770701,20180109,314,48.8282,9.2000,Stuttgart (Schnarrenberg),Baden-Württemberg


In [4]:
class DWDBase(object):
    def __init__(self, station_ids):
        self.station_ids = station_ids
        self.pattrn = '(?:stundenwerte\_(\d|\w||_)+\.zip)'
        
    def download_products(self):
        all_urls = (self.infer_urls_for_stations(self.hist_base_url, self.station_ids) + 
                     self.infer_urls_for_stations(self.rece_base_url, self.station_ids))
        all_dfs = [df for df in [self.download_products_from_url(u) for u in all_urls] if df is not None]
        df = pd.concat(all_dfs)
        return self.postprocess_data(df)
            
    def infer_urls_for_stations(self, base_url, station_ids):
        with ureq.urlopen(base_url) as res:
            listing = [re.search(self.pattrn, f).group(0) for f in [p for p in 
                   [l.decode("utf-8") for l in res.readlines()] 
                   if re.search(self.pattrn, p)
            ]]
        files = [[l for l in listing if ("_%05d_" % d) in l] for d in station_ids]
        return [pth.join(base_url, f) for f in itertools.chain(*files)]
    
    def download_products_from_url(self, url):
        try:
            with ureq.urlopen(url) as resp:
                with zf.ZipFile(io.BytesIO(resp.read())) as z:
                    products = [f.filename for f in z.filelist if "produkt_" in f.filename]
                    if not len(products):
                        raise Exception("No products in zip")

                    with z.open(products[0]) as f:
                        df = pd.read_csv(f, header=0, sep=";")
                        return df
        except Exception as ex:
            print("Unable to extract product from '%s' due to %s" % (url, ex))
            return None
        
    def postprocess_data(self, df):
        if "eor" in df.columns:
            df = df.drop('eor', 1)
        
        if "MESS_DATUM" in df.columns:
            df["TIMESTAMP"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d%H")
        
        return df

class DWDTemp(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/air_temperature/recent/"
        

class DWDPrecipitation(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/precipitation/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/precipitation/recent/"
        
class DWDCloudiness(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/cloudiness/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/cloudiness/recent/"
        
class DWDSun(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/sun/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/sun/recent/"
        
class DWDWind(DWDBase):
    def __init__(self, station_ids):
        super().__init__(station_ids)
        self.hist_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/wind/historical/"
        self.rece_base_url = "ftp://ftp-cdc.dwd.de/pub/CDC/observations_germany/climate/hourly/wind/recent/"

In [5]:
def join_station_data(ts_df, stations_df):
    right_df = stations_df.drop("von_datum", 1) \
                          .drop("bis_datum", 1) \
                          .drop("Bundesland", 1)  
    return pd.merge(left=ts_df, right=right_df, left_on="STATIONS_ID", right_on="Stations_id") \
             .drop("Stations_id", 1)

### Temperatur
- QN_9 - Qualitätsniveau der nachfolgenden Spalten
- TT_TU - Lufttemperatur in 2m Höhe in °C
- RF_TU - relative Feuchte in % 

#### Qualitaetsinformation
Die hier abgegebenen Qualitätsniveaus gelten jeweils für die nachfolgenden Spalten. Das Qualitätsniveau
(QN) beschreibt das Verfahren der Qualitätsprüfung und bezieht sich auf einen vollständigen Satz von
Parametern zu einem bestimmten Termin. Die einzelnen Parameter eines vollständigen Satzes sind
in der internen DWD-Datenbank mit jeweiligen Qualitätsbytes verknüpft, die hier nicht mit ausgegeben
werden. Als falsch oder zweifelhaft markierte Werte sind hier auf -999 gesetzt worden. Verschiedene
Qualitätsprüfverfahren (auf verschiedenen Stufen) entscheiden, welche Werte falsch oder zweifelhaft sind. In
der Vergangenheit wurden zum Teil andere Verfahren benutzt.
Qualitätsniveau (z.B. QN_9)

- 1 - nur formale Prüfung
- 2 - nach individuellen Kriterien geprüft
- 3 - alte automatische Prüfung und Korrektur
- 5 - historische, subjektive Verfahren
- 7 - 2. Prüfung durchlaufen, vor Korrektur
- 8 - Qualitätsicherung ausserhalb ROUTINE
- 9 - nicht alle Parameter korrigiert
- 10 - Qualitätsprüfung abgeschlossen, Korrekturen beendet

In [6]:
temp = DWDTemp(stgt_stats_df["Stations_id"].values)
temp_df = join_station_data(temp.download_products(), stgt_stats_df)

In [7]:
temp_df.sample(n=10)

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
131077,3278,2010031222,7,0.3,83.0,2010-03-12 22:00:00,355,48.5376,9.2733,Metzingen
517471,4294,1985122920,7,-2.7,74.0,1985-12-29 20:00:00,360,48.4704,8.9687,Rottenburg-Kiebingen
1096760,4928,1998051017,10,26.2,23.0,1998-05-10 17:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
473382,4160,2016122201,3,-4.5,100.0,2016-12-22 01:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof
1864963,6275,2016062408,7,27.1,60.0,2016-06-24 08:00:00,325,48.6705,9.4627,Notzingen
1058827,4928,1993121102,7,4.7,85.0,1993-12-11 02:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
1738880,4933,1973080614,5,29.8,40.0,1973-08-06 14:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
31298,2074,2007122712,7,1.2,75.0,2007-12-27 12:00:00,522,48.3751,8.9801,Hechingen
632769,4349,2007043007,7,10.8,50.0,2007-04-30 07:00:00,248,48.9569,9.0710,Sachsenheim
444954,4160,2014031813,7,17.0,38.0,2014-03-18 13:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof


### Niederschlagshöhe
- QN_8 - Qualitätsniveau der nachfolgenden Spalten
- R1 stündliche Niederschlagshöhe in mm
- RS_IND - Indikator ob Niederschlag gefallen (0 = kein Niederschlag, 1 = Niederschlag)
- WRTR - WR-Niederschlagsform (siehe Unten)

#### WR-Codierung
- -999  - Fehlwerte sind mit -999 gekennzeichnet
- 0 - kein Niederschlag gefallenen und/ oder keine Niederschlagshöhe aus abgesetzten Niederschlägen (wie Tau, Reif); 
- 1 - Niederschlagshöhe ausschließlich aus abgesetzten Niederschlägen (fest und flüssig) oder es kann nicht zwischen fest und flüssig unterschieden werden; 
- 2 - Niederschlagshöhe ausschließlich aus flüssigen abgesetzten Niederschlägen ("weisser Tau" wird den flüssigen abgesetzten Niederschlägen zugeordnet) ; 
- 3 - Niederschlagshöhe ausschließlich aus festen abgesetzten Niederschlägen; 
- 6 - gefallener Niederschlag nur in flüssiger Form, kann auch abgesetzten Niederschlag jeder Art enthalten; 
- 7 - gefallener Niederschlag nur in fester Form, kann auch abgesetzten Niederschlag jeder Art enthalten; 
- 8 - gefallener Niederschlag in fester und flüssiger Form, kann auch abgesetzten Niederschlag jeder Art enthalten;
- 9 - Niederschlagsmessung ausgefallen, die Niederschlagsform kann nicht festgestellt werden.

In [8]:
pert = DWDPrecipitation(stgt_stats_df["Stations_id"].values)
pert_df = join_station_data(pert.download_products(), stgt_stats_df)

In [9]:
pert_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_8,R1,RS_IND,WRTR,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
442789,4160,2010102004,3,0.0,0,0,2010-10-20 04:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof
1213596,4931,2017070611,3,0.0,0,0,2017-07-06 11:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
856788,4928,1999110121,1,0.0,0,-999,1999-11-01 21:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
1237647,6275,2006092217,3,0.0,0,0,2006-09-22 17:00:00,325,48.6705,9.4627,Notzingen
143458,3278,2005052213,3,0.0,0,0,2005-05-22 13:00:00,355,48.5376,9.2733,Metzingen
707232,4349,2013031116,3,0.1,1,-999,2013-03-11 16:00:00,248,48.9569,9.0710,Sachsenheim
829049,4926,2011072218,3,0.0,0,-999,2011-07-22 18:00:00,224,48.7896,9.2167,Stuttgart (Neckartal)
60651,2074,2011050317,3,0.0,0,0,2011-05-03 17:00:00,522,48.3751,8.9801,Hechingen
498302,4160,2016082705,3,0.0,0,-999,2016-08-27 05:00:00,478,48.7425,8.9240,Renningen-Ihinger Hof
1170378,4931,2013012217,3,0.0,1,7,2013-01-22 17:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen


### Wolkenbedeckung
- QN_9 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- V_N_I - Index wie Messung erhoben wurde (P = vom Beobachter, I = vom Instrument)
- V_N - Bedeckungsgrad aller Wolken auf Skala 1-8 (nicht erkennbar -1)

In [10]:
cloud = DWDCloudiness(stgt_stats_df["Stations_id"].values)
cloud_df = join_station_data(cloud.download_products(), stgt_stats_df)

In [11]:
cloud_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_8,V_N_I,V_N,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
123439,4928,1990122100,1,P,8,1990-12-21 00:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
326618,4931,1958090223,1,P,0,1958-09-02 23:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
334839,4931,1959081112,1,P,7,1959-08-11 12:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
579130,4931,1994040817,1,P,4,1994-04-08 17:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
602033,4931,1996111900,1,P,8,1996-11-19 00:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
336441,4931,1959101706,1,P,1,1959-10-17 06:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
553456,4931,1991042823,1,P,8,1991-04-28 23:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
136220,4928,1994062415,1,P,0,1994-06-24 15:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
715544,4931,2009110401,3,P,8,2009-11-04 01:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
718612,4931,2010031121,3,P,7,2010-03-11 21:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen


### Sonnenscheindauer
- QN_7 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- SD_SO - Stündliche Sonnenscheindauer

In [12]:
sun = DWDSun(stgt_stats_df["Stations_id"].values)
sun_df = join_station_data(sun.download_products(), stgt_stats_df)

In [13]:
sun_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_7,SD_SO,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
1294734,4931,2006122716,10,21.0,2006-12-27 16:00:00,371,48.6883,9.2235,Stuttgart-Echterdingen
1386454,4933,1953051708,5,60.0,1953-05-17 08:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
336517,3402,2016061417,10,2.0,2016-06-14 17:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
332002,3402,2015100720,10,0.0,2015-10-07 20:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
902755,4927,1973091920,5,0.0,1973-09-19 20:00:00,286,48.7693,9.1814,Stuttgart-Stadt
775349,4927,1954050418,5,0.0,1954-05-04 18:00:00,286,48.7693,9.1814,Stuttgart-Stadt
1431962,4933,1960041812,5,0.0,1960-04-18 12:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
1015465,4928,1987041314,7,0.0,1987-04-13 14:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
100716,3278,2005042119,10,0.0,2005-04-21 19:00:00,355,48.5376,9.2733,Metzingen
186012,3402,1951073113,5,60.0,1951-07-31 13:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten


### Windgeschwindigkeit und Windrichtung
- QN_3 - Qualitätsniveau der nachfolgenden Spalten (siehe Temperatur)
- F - mittlere Windgeschwindigkeit in m/s
- D - mittlere Windrichtung in Grad

In [14]:
wind = DWDWind(stgt_stats_df["Stations_id"].values)
wind_df = join_station_data(wind.download_products(), stgt_stats_df)

In [15]:
wind_df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_3,F,D,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname
708817,4928,2012030401,10,3.4,310,2012-03-04 01:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
195983,4927,1961071222,5,0.7,-999,1961-07-12 22:00:00,286,48.7693,9.1814,Stuttgart-Stadt
324817,4927,1976032400,5,1.0,190,1976-03-24 00:00:00,286,48.7693,9.1814,Stuttgart-Stadt
62989,3402,2015041401,10,1.3,330,2015-04-14 01:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten
155088,4927,1956111123,5,1.8,-999,1956-11-11 23:00:00,286,48.7693,9.1814,Stuttgart-Stadt
1331234,4933,1969032114,5,4.5,-999,1969-03-21 14:00:00,401,48.7091,9.2147,Stuttgart-Hohenheim
360897,4927,1980050508,5,2.6,360,1980-05-05 08:00:00,286,48.7693,9.1814,Stuttgart-Stadt
117338,4927,1952072301,5,0.7,-999,1952-07-23 01:00:00,286,48.7693,9.1814,Stuttgart-Stadt
707486,4928,2012010814,10,5.5,290,2012-01-08 14:00:00,314,48.8282,9.2000,Stuttgart (Schnarrenberg)
11155,3402,2009050910,10,3.5,140,2009-05-09 10:00:00,750,48.3851,9.4837,Münsingen-Apfelstetten


In [3]:
sess = SparkSession.builder \
                   .master("local[*]") \
                   .config("spark.driver.memory", "16g") \
                   .getOrCreate()

In [23]:
def convert_datetime64_columns(df):
    for column_name, column in df.iteritems():
        if column.dtype.kind == 'M':
            df[column_name] = pd.Series(column.dt.to_pydatetime(), dtype=object)
    return df

def sanitize_column_name(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s)

def sanitize_all_columns(df):
    for orig_col in df.columns:
        df = df.withColumnRenamed(orig_col, sanitize_column_name(orig_col))
    return df

def transform_pd_to_spark_and_write_parquet(pd_df, parquet_path, pipeline=None):
    pd_df = convert_datetime64_columns(pd_df)
    s_df = sanitize_all_columns(sess.createDataFrame(pd_df))
    if pipeline:
        model = pipeline.fit(s_df)
        s_df = model.transform(s_df)
    s_df.write.mode("overwrite").parquet(parquet_path)

In [19]:
ts_trans = TimestampTransformer(
    inputCol="TIMESTAMP",
    outputCol="datedim"
)

gh_trans = GeohashTransformer(
    inputCols=["geoBreite", "geoLaenge"],
    outputCol="geohash"
)

pipeline = Pipeline(stages=[ts_trans, gh_trans])

In [1]:
transform_pd_to_spark_and_write_parquet(temp_df, "./dwd/temp.parquet", pipeline)
transform_pd_to_spark_and_write_parquet(pert_df, "./dwd/precipitation.parquet", pipeline)
transform_pd_to_spark_and_write_parquet(cloud_df, "./dwd/cloudiness.parquet", pipeline)
transform_pd_to_spark_and_write_parquet(sun_df, "./dwd/sun.parquet", pipeline)
transform_pd_to_spark_and_write_parquet(wind_df, "./dwd/wind.parquet", pipeline)

NameError: name 'transform_pd_to_spark_and_write_parquet' is not defined

In [4]:
s_temp_df = sess.read.parquet("./dwd/temp.parquet")
s_temp_df.printSchema()
print(s_temp_df.count())

root
 |-- STATIONS_ID: long (nullable = true)
 |-- MESS_DATUM: long (nullable = true)
 |-- QN_9: long (nullable = true)
 |-- TT_TU: double (nullable = true)
 |-- RF_TU: double (nullable = true)
 |-- TIMESTAMP: timestamp (nullable = true)
 |-- Stationshoehe: long (nullable = true)
 |-- geoBreite: double (nullable = true)
 |-- geoLaenge: double (nullable = true)
 |-- Stationsname: string (nullable = true)
 |-- datedim: struct (nullable = true)
 |    |-- year: integer (nullable = true)
 |    |-- month: integer (nullable = true)
 |    |-- day: integer (nullable = true)
 |    |-- day_of_week: integer (nullable = true)
 |    |-- weekend: integer (nullable = true)
 |    |-- holiday: integer (nullable = true)
 |    |-- day_cat: string (nullable = true)
 |    |-- day_num: double (nullable = true)
 |    |-- day_cos: double (nullable = true)
 |    |-- day_sin: double (nullable = true)
 |    |-- time_cat: string (nullable = true)
 |    |-- time_bin: integer (nullable = true)
 |    |-- hour_bin: in

In [5]:
s_temp_df.limit(100).toPandas()

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,TIMESTAMP,Stationshoehe,geoBreite,geoLaenge,Stationsname,datedim,geohash
0,4928,1980070418,5,18.3,49.0,1980-07-04 18:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.67931547619047...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
1,4928,1980070419,5,17.9,48.0,1980-07-04 19:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.68526785714285...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
2,4928,1980070420,5,16.9,53.0,1980-07-04 20:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.69122023809523...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
3,4928,1980070421,5,15.1,70.0,1980-07-04 21:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.69717261904761...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
4,4928,1980070422,5,15.0,71.0,1980-07-04 22:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.703125, -0.290...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
5,4928,1980070423,5,14.7,72.0,1980-07-04 23:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 4, 4, 0, 0, Friday, 0.70907738095238...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
6,4928,1980070500,5,14.1,74.0,1980-07-05 00:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 5, 5, 1, 0, Saturday, 0.715029761904...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
7,4928,1980070501,5,13.9,75.0,1980-07-05 01:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 5, 5, 1, 0, Saturday, 0.720982142857...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
8,4928,1980070502,5,13.2,82.0,1980-07-05 02:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 5, 5, 1, 0, Saturday, 0.726934523809...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
9,4928,1980070503,5,12.9,84.0,1980-07-05 03:00:00,314,48.8282,9.2,Stuttgart (Schnarrenberg),"(1980, 7, 5, 5, 1, 0, Saturday, 0.732886904761...","(u0wtc2ghzs1n, 15004188053247755086, u0wtc2gh)"
